# Neptune Connection Testing

This notebook tests direct connection to Neptune following the official documentation.

In [ ]:
import os
import boto3
import logging
from botocore.auth import SigV4Auth
from botocore.awsrequest import AWSRequest
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection
from gremlin_python.driver.protocol import GremlinServerError
from gremlin_python.process.anonymous_traversal import traversal
from gremlin_python.driver.tornado.transport import TornadoTransport

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

## 1. Connection Configuration

In [ ]:
# Neptune endpoint
ENDPOINT = "graph-rag-originofcovid19dataset-benchmark.cluster-c7m8ay28gj4o.us-west-2.neptune.amazonaws.com"
PORT = 8182

# Construct database URL
database_url = f"wss://{ENDPOINT}:{PORT}/gremlin"
print(f"Database URL: {database_url}")

## 2. Test Connection

In [ ]:
def create_remote_connection(url, headers=None):
    """Create a remote connection with Tornado transport."""
    return DriverRemoteConnection(
        url,
        'g',
        headers=headers,
        transport_factory=lambda: TornadoTransport(
            read_timeout=300,
            write_timeout=300
        )
    )

try:
    # Get AWS credentials
    print("Getting AWS credentials...")
    creds = boto3.Session().get_credentials().get_frozen_credentials()
    
    # Set up request with IAM auth
    print("Setting up IAM auth...")
    request = AWSRequest(method="GET", url=database_url)
    SigV4Auth(creds, "neptune-db", boto3.Session().region_name).add_auth(request)
    
    # Initialize connection
    print("Initializing connection...")
    remoteConn = create_remote_connection(
        database_url,
        headers=request.headers.items()
    )
    
    # Create traversal source
    print("Creating traversal source...")
    g = traversal().withRemote(remoteConn)
    
    # Test simple query
    print("\nTesting query...")
    result = g.inject(1).toList()  # Simple test query that doesn't require graph access
    print(f"Query result: {result}")
    
    print("\nConnection test successful!")
    
except Exception as e:
    print(f"\nConnection failed: {str(e)}")
    raise
    
finally:
    # Clean up
    if 'remoteConn' in locals():
        print("\nClosing connection...")
        remoteConn.close()